In [9]:
# knnMatch 함수로부터 올바른 매칭점 찾기 (match_good_knn.py)

import cv2, numpy as np
import imutils
import pandas as pd

photo_img_path = "AutoAlign_test/B17545___________000_lat_photo.jpg"
film_img_path = "AutoAlign_test/B17545___________000_lat_film.jpg"
film_tsv_path = "AutoAlign_test/B17545___________000_lat_film.txt"

landmark_regex_string = '29@[2479]|30@[34]'
landmark_number = 6

photo = cv2.imread(photo_img_path)
film = cv2.imread(film_img_path, cv2.IMREAD_GRAYSCALE)

In [10]:
def show_image(image, size=(800, 600)):
    cv2.namedWindow('border', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('border', size[0], size[1])

    cv2.imshow('border', image)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [11]:
def parse_tsv(tsv_path: str):
    '''
    parse_tsv(tsv_path: str):
        tsv_path: str: full path to text file, in tab separated values format.

    Opens text file in tsv_path as tsv. loads it as pandas dataframe, in a following format:

    |   name    |   X   |   Y   |
    =============================
    |   ...     |  ...  |  ...  |
    |  '29@2'   | 324.7 | 250.4 |
    |   (str)   | float | float |
    |   ...     |  ...  |  ...  |

    If fails, prints error message and quits.

    return:
        df: Pandas.dataframe() that contains landmark information.
    '''
    # Loading dataframe

    df = pd.read_csv(tsv_path,  sep='\t')
    df = df.iloc[:99, 0:3]
    
    df.columns = ['name', 'X', 'Y']

    return df


def extract_landmarks(df, landmark_regex, landmark_length):
    '''
    extract_landmarks(df: pandas.dataframe(), landmark_regex: str, landmark_length: int):
        df: Pandas.dataframe() that contains landmark information.
        landmark_regex: Regular Expression String that matches certain landmark names.
        landmark_length: length of landmarks(how many landmarks do we want to extract?).

    Gathers needed landmarks, sort, drop name, and change it into numpy array.

    returns:
        landmark: numpy.array(): (landmark_length, 2) shaped numpy array with landmark(x, y) in each row.
    '''
    # (gathering only needed landmarks)
    df = df.loc[df['name'].str.contains(landmark_regex, regex=True), :]
    # there are **18** landmarks that is unique and valid among all files
    df = df.sort_values(by=['name'])
    df = df.loc[:, ['X', 'Y']]
    df = df.reset_index(drop=True)

    # ... and landmark
    landmark = df.to_numpy(dtype=np.float32)
    return landmark

In [12]:
border = imutils.auto_canny(photo)

# show_image(border)

In [14]:
film_landmarks = extract_landmarks(parse_tsv(film_tsv_path), landmark_regex_string, landmark_number)
display(film_landmarks)

array([[1648.,  872.],
       [1846., 1255.],
       [1737., 1388.],
       [1782., 1555.],
       [1729., 1649.],
       [1610., 1738.]], dtype=float32)

In [35]:
film2 = imutils.adjust_brightness_contrast(film, 0., 50)
margin = 32
box = np.array([[min(film_landmarks[:, 0])-margin, min(film_landmarks[:, 1])-margin], [max(film_landmarks[:, 0])+margin, max(film_landmarks[:, 1])+margin]], dtype=int)
print(box)

crop = film2[box[0,1]:box[1,1],box[1,0]-500:box[1,0]]
(thresh, im_bw) = cv2.threshold(crop, 20, 255, cv2.THRESH_BINARY)


canny2 = imutils.auto_canny(im_bw)
show_image(canny2)




[[1578  840]
 [1878 1770]]


In [28]:
def find_fft(img):
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    return fshift

In [29]:
max_y, max_x = max(zip(*np.nonzero(border)), key=lambda c: c[1])
border2 = border[max_y-300:max_y+300, max_x-300:max_x+margin]
show_image(border2)

In [39]:

import imreg_dft
print(border2.shape)
print(canny2.shape)
comp1 = cv2.resize(border2, (600,300))
comp2 = cv2.resize(canny2, (600,300))
show_image(comp1)
show_image(comp2)
result  = imreg_dft.imreg.similarity(comp1, comp2)
show_image(result["timg"])


(600, 332)
(930, 500)


In [20]:
# ORB로 서술자 추출 ---①
detector = cv2.ORB_create()
kp1, desc1 = detector.detectAndCompute(border2, None)
kp2, desc2 = detector.detectAndCompute(canny2, None)
# BF-Hamming 생성 ---②
matcher = cv2.BFMatcher(cv2.NORM_HAMMING2)
# knnMatch, k=2 ---③
matches = matcher.knnMatch(desc1, desc2, 2)

# 첫번재 이웃의 거리가 두 번째 이웃 거리의 75% 이내인 것만 추출---⑤
ratio = 0.75
good_matches = [first for first,second in matches \
                    if first.distance < second.distance * ratio]
print('matches:%d/%d' %(len(good_matches),len(matches)))

# 좋은 매칭만 그리기
res = cv2.drawMatches(border2, kp1, canny2, kp2, good_matches, None, \
                    flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
# 결과 출력                    
show_image(res)

matches:25/350


In [21]:

# 좋은 매칭점의 queryIdx로 원본 영상의 좌표 구하기 ---③
src_pts = np.float32([ kp1[m.queryIdx].pt for m in good_matches ])
# 좋은 매칭점의 trainIdx로 대상 영상의 좌표 구하기 ---④
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good_matches ])
mtrx, mask = cv2.findHomography(src_pts, dst_pts)
print(mtrx)
h,w, = border2.shape[:2]
pts = np.float32([ [[0,0]],[[0,h-1]],[[w-1,h-1]],[[w-1,0]] ])
# 원본 영상 좌표를 원근 변환  ---⑦
print(border2.shape)
print(pts)
dst = cv2.perspectiveTransform(pts,mtrx)
print(canny2.shape)
print([np.int32(dst)])

# 변환 좌표 영역을 대상 영상에 그리기 ---⑧
canny2 = cv2.polylines(canny2,np.int32(dst),True,255,3, cv2.LINE_AA)


show_image(canny2)

# 좋은 매칭 그려서 출력 ---⑨
res = cv2.drawMatches(border2, kp1, canny2, kp2, good_matches, None, \
                    flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
cv2.imshow('Matching Homography', res)
cv2.waitKey()
cv2.destroyAllWindows()

[[-8.43880405e-01 -5.84283184e-03  1.89575319e+02]
 [-2.35078103e+00  3.54154505e-04  5.24101552e+02]
 [-4.47602480e-03 -1.74727845e-05  1.00000000e+00]]
(600, 332)
[[[  0.   0.]]

 [[  0. 599.]]

 [[331. 599.]]

 [[331.   0.]]]
(930, 300)
[array([[[189, 524]],

       [[188, 529]],

       [[189, 515]],

       [[186, 527]]])]
